In [ ]:
# !apt update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
# !tar -xvf spark-3.3.0-bin-hadoop3.tgz
# !pip install -q findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [1]:
import findspark
findspark.init()

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [3]:
# %cd '/content/gdrive/MyDrive/Data_Sicience/LDS0_K282_ONLINE_LuuNguyenKhaiHoan/Project_2/Data'

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [5]:
%cd "C:\Users\khaih\Downloads\Project_2\Data

C:\Users\khaih\Downloads\Project_2\Data


In [6]:
SparkContext.setSystemProperty('spark.executor.memory', '8g')
sc = SparkContext(master='local', appName='Project_2_ALS')

In [7]:
spark = SparkSession(sc)

###Read Data

In [8]:
import pandas as pd
#data = pd.read_csv("pre_data.csv", ,index_col=0)
products= pd.read_csv("pre_products.csv",delimiter=',',index_col=0)
reviews=pd.read_csv("ReviewRaw.csv",delimiter=',')

In [9]:
products.head()

,index,item_id,rating,product_content,product_content_wt
0,0,48102821,4.0,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,vân chống nước nhạc từ sạc đầy chỉ khoảng chờ ...
1,1,52333193,4.5,Tai nghe bluetooth không dây F9 True wireless ...,không dây wireless có báo kép nhạc giờ giờ . o...
2,2,299461,4.8,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,chuột không dây hàng chính hãng thương hiệu cá...
3,3,57440329,4.7,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,robot hàng chính hãng acome sku chính hãng aco...
4,4,38458616,4.8,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,hàng chính hãng mỹ hộp sạc sạc sku và nhỏ gọn ...


In [10]:
reviews.head()

,customer_id,product_id,name,full_name,created_time,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,Lân Nguyễn Hoàng,NaN,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,Nguyễn Khánh Hòa,NaN,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,Toàn Phạm Khánh,2019-04-17 15:42:45,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,NaN,NaN,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,Phạm Bá Đức,NaN,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."


In [11]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364099 entries, 0 to 364098
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   364099 non-null  int64 
 1   product_id    364099 non-null  int64 
 2   name          363669 non-null  object
 3   full_name     329471 non-null  object
 4   created_time  117340 non-null  object
 5   rating        364099 non-null  int64 
 6   title         364070 non-null  object
 7   content       165794 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.2+ MB


In [12]:
reviews.isnull().sum()

customer_id          0
product_id           0
name               430
full_name        34628
created_time    246759
rating               0
title               29
content         198305
dtype: int64

In [13]:
reviews.duplicated().sum()

2349

In [14]:
reviews=reviews.drop_duplicates()

In [15]:
reviews.duplicated().sum()

0

In [16]:
stringcols = reviews.select_dtypes(include='object').columns
reviews[stringcols] = reviews[stringcols].astype("str")

In [17]:
data_sub=spark.createDataFrame(reviews) 
data_sub.printSchema()

C:\spark\spark-3.3.1-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\spark\spark-3.3.1-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- customer_id: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- created_time: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- title: string (nullable = true)
 |-- content: string (nullable = true)



###Clean and Prepare the Data

In [18]:
data_sub.show(3)

+-----------+----------+----------------+----------------+-------------------+------+------------------+--------------------+
|customer_id|product_id|            name|       full_name|       created_time|rating|             title|             content|
+-----------+----------+----------------+----------------+-------------------+------+------------------+--------------------+
|     709310|  10001012|Lân Nguyễn Hoàng|Lân Nguyễn Hoàng|                nan|     3|Ko dùng đc thẻ nhớ|Lúcđầu quên thông...|
|   10701688|  10001012|Nguyễn Khánh Hòa|Nguyễn Khánh Hòa|                nan|     5|   Cực kì hài lòng|Tiki giao hàng nh...|
|   11763074|  10001012| Toàn Phạm Khánh| Toàn Phạm Khánh|2019-04-17 15:42:45|     5|   Cực kì hài lòng|chất lượng camera...|
+-----------+----------+----------------+----------------+-------------------+------+------------------+--------------------+
only showing top 3 rows



In [19]:
data_sub = data_sub.select([ 'customer_id','product_id', 'rating'])

In [20]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import isnan, when, count, col, udf

In [21]:
data_sub = data_sub.withColumn("rating", data_sub["rating"].cast(DoubleType()))

In [22]:
data_sub.show(5, truncate=True)

+-----------+----------+------+
|customer_id|product_id|rating|
+-----------+----------+------+
|     709310|  10001012|   3.0|
|   10701688|  10001012|   5.0|
|   11763074|  10001012|   5.0|
|    9909549|  10001012|   5.0|
|    1827148|  10001012|   5.0|
+-----------+----------+------+
only showing top 5 rows



In [23]:
data_sub.select([count(when(col(c).isNull(), c)).alias(c) for c in 
           data_sub.columns]).toPandas().T

,0
customer_id,0
product_id,0
rating,0


In [24]:
users = data_sub.select("customer_id").distinct().count()
products = data_sub.select("product_id").distinct().count()
numerator = data_sub.count()

In [25]:
display(numerator, users, products)

361750

251491

4218

In [26]:
denominator = users * products
denominator

1060789038

In [27]:
sparsity = 1 - (numerator*1.0 / denominator)
print ("Sparsity: "), sparsity

Sparsity: 


(None, 0.9996589802618228)

In [28]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [29]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import numpy as np

###Feature Transformations

In [30]:
indexer = StringIndexer(inputCol='product_id', 
                        outputCol='product_id_idx')

indexer_model = indexer.fit(data_sub)

data_indexed = indexer_model.transform(data_sub)

indexer1 = StringIndexer(inputCol='customer_id', 
                         outputCol='customer_id_idx')
indexer1_model = indexer1.fit(data_indexed)
data_indexed = indexer1_model.transform(data_indexed)

In [31]:
data_indexed.show(5, truncate=True)

+-----------+----------+------+--------------+---------------+
|customer_id|product_id|rating|product_id_idx|customer_id_idx|
+-----------+----------+------+--------------+---------------+
|     709310|  10001012|   3.0|        2458.0|       216812.0|
|   10701688|  10001012|   5.0|        2458.0|        24473.0|
|   11763074|  10001012|   5.0|        2458.0|        26843.0|
|    9909549|  10001012|   5.0|        2458.0|         3377.0|
|    1827148|  10001012|   5.0|        2458.0|          396.0|
+-----------+----------+------+--------------+---------------+
only showing top 5 rows



In [32]:
data_indexed.select([count(when(col(c).isNull(), c)).alias(c) for c in 
           data_indexed.columns]).toPandas().T

,0
customer_id,0
product_id,0
rating,0
product_id_idx,0
customer_id_idx,0


Chia train, test với tỉ lệ 8/2

In [33]:
(training, test) = data_indexed.randomSplit([0.8, 0.2])

###The Model

In [34]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from time import time

In [35]:
t0 = time()
als = ALS(maxIter=10, 
          regParam=0.3,           
          rank = 20,
          userCol="customer_id_idx", 
          itemCol="product_id_idx", 
          ratingCol="rating", 
          coldStartStrategy="drop",
          nonnegative=True)
model = als.fit(training)
t1 = time()

In [36]:
# import joblib

# joblib.dump(als, "als.joblib")

In [37]:
total_time = t1 - t0

In [38]:
total_time

73.11907815933228

### Combine results

In [39]:
predictions = model.transform(test)

In [40]:
predictions.show(10)

+-----------+----------+------+--------------+---------------+----------+
|customer_id|product_id|rating|product_id_idx|customer_id_idx|prediction|
+-----------+----------+------+--------------+---------------+----------+
|    7377207|   3524487|   4.0|         148.0|            3.0| 3.5237927|
|       5235|   2902221|   5.0|         833.0|           22.0|  4.450485|
|    6329070|   2902221|   5.0|         833.0|           53.0| 3.8573759|
|    5916266|   3524487|   3.0|         148.0|          183.0| 3.5808241|
|    7206266|    416985|   5.0|         471.0|          331.0| 4.1404486|
|    7863386|   3524487|   5.0|         148.0|          336.0|  2.955128|
|     232742|  13525399|   5.0|        2366.0|          414.0|  4.775419|
|    8300548|   2902221|   3.0|         833.0|          472.0|  4.366674|
|     941861|  32591390|   5.0|         496.0|          484.0| 4.2079678|
|   18973337|  75754994|   5.0|        1829.0|          565.0|  4.257287|
+-----------+----------+------+-------

In [41]:
evaluator = RegressionEvaluator(metricName="rmse", 
                                labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [42]:
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.14548579921352


### CrossValidator

In [43]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [44]:
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 20, 30]) \
            .addGrid(als.regParam, [.05, .1, .15, 0.2, 0.25, 0.3]) \
            .build()

In [45]:
crossvalidation = CrossValidator(estimator = als, estimatorParamMaps = param_grid, evaluator = evaluator, numFolds=5)

In [46]:
model = crossvalidation.fit(training)

In [47]:
#Extract best model from the cv model above
best_model = model.bestModel

In [48]:
print("*Best Model*")
print("  Rank:", best_model._java_obj.parent().getRank(),"-  MaxIter:", best_model._java_obj.parent().getMaxIter(), "-  RegParam:", best_model._java_obj.parent().getRegParam())

*Best Model*
  Rank: 10 -  MaxIter: 10 -  RegParam: 0.3


In [49]:
print("RMSE value after cross validation is: ", evaluator.evaluate(best_model.transform(test)))

RMSE value after cross validation is:  1.138594531085801


### Run Model với best params

In [50]:
t0 = time()
als = ALS(maxIter=10, 
          regParam=0.3,           
          rank = 10,
          userCol="customer_id_idx", 
          itemCol="product_id_idx", 
          ratingCol="rating", 
          coldStartStrategy="drop",
          nonnegative=True)
model = als.fit(training)
t1 = time()

In [51]:
total_time = t1 - t0

In [52]:
total_time

62.53958177566528

In [53]:
predictions = model.transform(test)

In [54]:
predictions.show(10)

+-----------+----------+------+--------------+---------------+----------+
|customer_id|product_id|rating|product_id_idx|customer_id_idx|prediction|
+-----------+----------+------+--------------+---------------+----------+
|    7377207|   3524487|   4.0|         148.0|            3.0| 3.3931534|
|       5235|   2902221|   5.0|         833.0|           22.0| 4.3867836|
|    6329070|   2902221|   5.0|         833.0|           53.0| 4.2630324|
|    5916266|   3524487|   3.0|         148.0|          183.0| 3.6790729|
|    7206266|    416985|   5.0|         471.0|          331.0| 4.1553154|
|    7863386|   3524487|   5.0|         148.0|          336.0|  2.788553|
|     232742|  13525399|   5.0|        2366.0|          414.0|  4.837216|
|    8300548|   2902221|   3.0|         833.0|          472.0| 4.4437437|
|     941861|  32591390|   5.0|         496.0|          484.0|  4.134396|
|   18973337|  75754994|   5.0|        1829.0|          565.0|  4.188925|
+-----------+----------+------+-------

In [55]:
evaluator = RegressionEvaluator(metricName="rmse", 
                                labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [56]:
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.138594531085801


In [58]:
model.write().save("Project_2_ALS_model_1")

### Recommend For AllUsers

In [59]:
#user_recs = model.recommendForAllUsers(5) 
user_recs=model.recommendForAllUsers(3)

In [60]:
user_recs.show(5, truncate=False)

+---------------+---------------------------------------------------------+
|customer_id_idx|recommendations                                          |
+---------------+---------------------------------------------------------+
|1              |[{3915, 5.377843}, {4031, 5.1479664}, {4204, 5.102781}]  |
|3              |[{3915, 4.8678513}, {4204, 4.79311}, {3816, 4.758471}]   |
|4              |[{3915, 5.735988}, {4204, 5.3854423}, {4049, 5.199909}]  |
|5              |[{3915, 6.0398607}, {4204, 5.4590397}, {4049, 5.2181854}]|
|6              |[{3915, 5.5245967}, {4204, 5.4235983}, {4049, 5.266618}] |
+---------------+---------------------------------------------------------+
only showing top 5 rows



In [61]:
user_recs.count()

211480

In [62]:
for user in user_recs.head(5):
    print(user.recommendations)
    print("\n")

[Row(product_id_idx=3915, rating=5.377842903137207), Row(product_id_idx=4031, rating=5.147966384887695), Row(product_id_idx=4204, rating=5.102780818939209)]


[Row(product_id_idx=3915, rating=4.867851257324219), Row(product_id_idx=4204, rating=4.793109893798828), Row(product_id_idx=3816, rating=4.7584710121154785)]


[Row(product_id_idx=3915, rating=5.735988140106201), Row(product_id_idx=4204, rating=5.38544225692749), Row(product_id_idx=4049, rating=5.199909210205078)]


[Row(product_id_idx=3915, rating=6.039860725402832), Row(product_id_idx=4204, rating=5.459039688110352), Row(product_id_idx=4049, rating=5.2181854248046875)]


[Row(product_id_idx=3915, rating=5.524596691131592), Row(product_id_idx=4204, rating=5.423598289489746), Row(product_id_idx=4049, rating=5.266617774963379)]




In [63]:
df_customer_id_customer_id_idx = data_indexed.select('customer_id_idx', 'customer_id').distinct()

In [64]:
df_customer_id_customer_id_idx.count()

251491

In [65]:
df_customer_id_customer_id_idx.show(5)

+---------------+-----------+
|customer_id_idx|customer_id|
+---------------+-----------+
|       101626.0|   13800460|
|       194826.0|    5493687|
|        47896.0|     560271|
|         5582.0|    8311997|
|        36226.0|    1671910|
+---------------+-----------+
only showing top 5 rows



In [66]:
df_product_id_product_id_idx = data_indexed.select('product_id_idx', 'product_id').distinct()

In [67]:
df_product_id_product_id_idx.count()

4218

In [68]:
df_product_id_product_id_idx.show(5)

+--------------+----------+
|product_id_idx|product_id|
+--------------+----------+
|        1635.0|  10001479|
|        2051.0|  12321426|
|        1876.0|  12477248|
|         263.0|  17929212|
|         636.0|  26314426|
+--------------+----------+
only showing top 5 rows



In [69]:
for user in user_recs.head(5):
    print(user)
    print("\n")

Row(customer_id_idx=1, recommendations=[Row(product_id_idx=3915, rating=5.377842903137207), Row(product_id_idx=4031, rating=5.147966384887695), Row(product_id_idx=4204, rating=5.102780818939209)])


Row(customer_id_idx=3, recommendations=[Row(product_id_idx=3915, rating=4.867851257324219), Row(product_id_idx=4204, rating=4.793109893798828), Row(product_id_idx=3816, rating=4.7584710121154785)])


Row(customer_id_idx=4, recommendations=[Row(product_id_idx=3915, rating=5.735988140106201), Row(product_id_idx=4204, rating=5.38544225692749), Row(product_id_idx=4049, rating=5.199909210205078)])


Row(customer_id_idx=5, recommendations=[Row(product_id_idx=3915, rating=6.039860725402832), Row(product_id_idx=4204, rating=5.459039688110352), Row(product_id_idx=4049, rating=5.2181854248046875)])


Row(customer_id_idx=6, recommendations=[Row(product_id_idx=3915, rating=5.524596691131592), Row(product_id_idx=4204, rating=5.423598289489746), Row(product_id_idx=4049, rating=5.266617774963379)])




In [70]:
new_user_recs = user_recs.join(df_customer_id_customer_id_idx, on=['customer_id_idx'], how='left')

In [71]:
new_user_recs.show(5, truncate=False)

+---------------+--------------------------------------------------------+-----------+
|customer_id_idx|recommendations                                         |customer_id|
+---------------+--------------------------------------------------------+-----------+
|7              |[{3915, 5.950756}, {4204, 5.4772243}, {4049, 5.3185005}]|6177374    |
|1              |[{3915, 5.377843}, {4031, 5.1479664}, {4204, 5.102781}] |7280719    |
|4              |[{3915, 5.735988}, {4204, 5.3854423}, {4049, 5.199909}] |1064154    |
|3              |[{3915, 4.8678513}, {4204, 4.79311}, {3816, 4.758471}]  |7377207    |
|6              |[{3915, 5.5245967}, {4204, 5.4235983}, {4049, 5.266618}]|1046981    |
+---------------+--------------------------------------------------------+-----------+
only showing top 5 rows



In [72]:
new_user_recs.count()

211480

In [73]:
new_user_recs.write.parquet('Project_2_ALS.parquet', mode='overwrite')
df_product_id_product_id_idx.write.parquet('Project_2_ALS.parquet', mode='overwrite')

In [74]:
userId = ['709310', '13800460', '5493687']
result_1 = new_user_recs.filter(new_user_recs["customer_id"]=='709310')
result_1.collect()

[Row(customer_id_idx=216812, recommendations=[Row(product_id_idx=3915, rating=3.281700372695923), Row(product_id_idx=4204, rating=3.009462833404541), Row(product_id_idx=3816, rating=2.9550321102142334)], customer_id=709310)]

In [75]:
result_2 = new_user_recs.filter(new_user_recs["customer_id"]==13800460)
result_2.show()

+---------------+--------------------+-----------+
|customer_id_idx|     recommendations|customer_id|
+---------------+--------------------+-----------+
|         101626|[{3915, 5.5620165...|   13800460|
+---------------+--------------------+-----------+



In [76]:
reviews.loc[reviews["customer_id"]==13800460]

,customer_id,product_id,name,full_name,created_time,rating,title,content
188,13800460,10001351,Trần Ngọc Lâm,Trần Ngọc Lâm,nan,5,rất hài lòng,khi mua được thông báo là hàng giao nhanh tron...


In [77]:
result_3 = new_user_recs.filter(new_user_recs["customer_id"]=='5493687')
result_3.show()

+---------------+--------------------+-----------+
|customer_id_idx|     recommendations|customer_id|
+---------------+--------------------+-----------+
|         194826|[{3915, 5.5148573...|    5493687|
+---------------+--------------------+-----------+



### Convert

In [78]:
import pandas as pd
recs=model.recommendForAllUsers(10).toPandas()
nrecs=recs.recommendations.apply(pd.Series) \
            .merge(recs, right_index = True, left_index = True) \
            .drop(["recommendations"], axis = 1) \
            .melt(id_vars = ['customer_id_idx'], value_name = "recommendation") \
            .drop("variable", axis = 1) \
            .dropna()
nrecs=nrecs.sort_values('customer_id_idx')
nrecs=pd.concat([nrecs['recommendation']\
              .apply(pd.Series), nrecs['customer_id_idx']], axis = 1)
nrecs.columns = [
          'product_id_idx',
          'rating',
          'customer_id_idx'
          ]

In [79]:
md=data_indexed.select(['customer_id', 'customer_id_idx', 'product_id', 'product_id_idx'])
md=md.toPandas()
dict1 =dict(zip(md['customer_id_idx'],md['customer_id']))
dict2=dict(zip(md['product_id_idx'],md['product_id']))
nrecs['customer_id']=nrecs['customer_id_idx'].map(dict1)
nrecs['product_id']=nrecs['product_id_idx'].map(dict2)
nrecs=nrecs.sort_values('customer_id')
nrecs.reset_index(drop=True, inplace=True)

In [80]:
new=nrecs[['customer_id','product_id','rating']]

In [81]:
new['recommendations'] = list(zip(new.product_id, new.rating))
res=new[['customer_id','recommendations']]
res_new=res['recommendations'].groupby([res.customer_id]).apply(list).reset_index()

C:\Users\khaih\AppData\Local\Temp\ipykernel_6228\44393553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['recommendations'] = list(zip(new.product_id, new.rating))


In [82]:
res_new

,customer_id,recommendations
0,10,"[(50592901, 4.914278984069824), (71293311, 5.0..."
1,27,"[(69625196, 5.523265838623047), (50592901, 4.9..."
2,28,"[(59081231, 4.805673122406006), (50592901, 4.8..."
3,36,"[(48520298, 4.986872673034668), (73844240, 5.0..."
4,50,"[(66251373, 4.87461519241333), (73844240, 4.94..."
...,...,...
211475,21003644,"[(76732229, 5.262803554534912), (48520298, 4.8..."
211476,21003802,"[(69201188, 4.120298862457275), (58186168, 4.1..."
211477,21007445,"[(50560427, 4.758530616760254), (48520298, 4.8..."
211478,21008104,"[(76732229, 5.2022294998168945), (48520298, 4...."


In [83]:
from operator import itemgetter

In [84]:
indices = range(len(res_new['recommendations'][0]))
df_=res_new['recommendations'].transform({f'sp{i+1}': itemgetter(i) for i in indices})
df_merged = pd.concat([res_new, df_], axis=1)

In [85]:
df_merged.head()

,customer_id,recommendations,sp1,sp2,sp3,sp4,sp5,sp6,sp7,sp8,sp9,sp10
0,10,"[(50592901, 4.914278984069824), (71293311, 5.0...","(50592901, 4.914278984069824)","(71293311, 5.019058704376221)","(69625196, 5.158987998962402)","(72969822, 4.902008056640625)","(76732229, 5.137398719787598)","(66251373, 5.000306606292725)","(19395453, 5.180264472961426)","(74558974, 4.860236644744873)","(73844240, 4.904170989990234)","(45651308, 4.862765312194824)"
1,27,"[(69625196, 5.523265838623047), (50592901, 4.9...","(69625196, 5.523265838623047)","(50592901, 4.941404819488525)","(72969822, 4.925102233886719)","(48520298, 4.91170597076416)","(19395453, 5.093387126922607)","(71293311, 5.1412353515625)","(76732229, 5.13853645324707)","(66251373, 5.088000297546387)","(50560427, 4.970122814178467)","(73844240, 4.9290266036987305)"
2,28,"[(59081231, 4.805673122406006), (50592901, 4.8...","(59081231, 4.805673122406006)","(50592901, 4.894715309143066)","(74558974, 4.803366184234619)","(45651308, 4.902621269226074)","(57421895, 4.81054162979126)","(72969822, 4.964327812194824)","(19395453, 4.865633010864258)","(71293311, 5.231959342956543)","(38120507, 4.8159589767456055)","(76732229, 5.084994316101074)"
3,36,"[(48520298, 4.986872673034668), (73844240, 5.0...","(48520298, 4.986872673034668)","(73844240, 5.043100833892822)","(72969822, 5.013724327087402)","(76732229, 5.3918962478637695)","(50592901, 5.1116533279418945)","(50560427, 5.135486125946045)","(71293311, 5.172455787658691)","(19395453, 5.147697448730469)","(2383179, 5.032731533050537)","(69625196, 5.606723785400391)"
4,50,"[(66251373, 4.87461519241333), (73844240, 4.94...","(66251373, 4.87461519241333)","(73844240, 4.94193696975708)","(50592901, 4.912436485290527)","(71293311, 4.895064830780029)","(76732229, 5.2437424659729)","(48520298, 4.863224029541016)","(73238633, 4.877793312072754)","(19395453, 5.0926361083984375)","(69625196, 5.49521541595459)","(50560427, 4.981415748596191)"


In [86]:
df_merged.to_csv("Project_2_ALS_1.csv")

### Viết Function in kết quả 

In [90]:
df_cus=pd.read_csv("Project_2_ALS_1.csv",index_col=0)
products_raw= pd.read_csv("ProductRaw.csv",delimiter=',')

In [95]:
def filter_customerID(df_merged,customer_id):
    index=df_merged[df_merged["customer_id"]==customer_id].index.values.astype(int)[0]
    sp=df_merged[df_merged["customer_id"]==customer_id].replace("\(", "", regex=True).replace("\)", "", regex=True).replace(" ", "", regex=True)
    list_product_ID=[]
    list_product_Rating=[]
    list_customerID=[]
    for i in range(1,11):
        index_="sp"+str(i)
        list_product_ID.append(sp[index_][index].split(",")[0])
        list_product_Rating.append(sp[index_][index].split(",")[1])
        list_customerID.append(customer_id)
    df=pd.DataFrame({"CustomerID":list_customerID,"Product ID":list_product_ID,"Rating":list_product_Rating})
    return df

### Kiểm tra kết quả với một vài Customer

In [96]:
filter_customerID(df_cus,10)

,CustomerID,Product ID,Rating
0,10,50592901,4.914278984069824
1,10,71293311,5.019058704376221
2,10,69625196,5.158987998962402
3,10,72969822,4.902008056640625
4,10,76732229,5.137398719787598
5,10,66251373,5.000306606292725
6,10,19395453,5.180264472961426
7,10,74558974,4.860236644744873
8,10,73844240,4.904170989990234
9,10,45651308,4.862765312194824


In [102]:
filter_customerID(df_cus,36)

,CustomerID,Product ID,Rating
0,36,48520298,4.986872673034668
1,36,73844240,5.043100833892822
2,36,72969822,5.013724327087402
3,36,76732229,5.3918962478637695
4,36,50592901,5.1116533279418945
5,36,50560427,5.135486125946045
6,36,71293311,5.172455787658691
7,36,19395453,5.147697448730469
8,36,2383179,5.032731533050537
9,36,69625196,5.606723785400391


In [97]:
from pyspark.ml.recommendation import ALSModel
model_2 = ALSModel.load('Project_2_ALS_model')

In [98]:
predictions = model_2.transform(test)

In [99]:
predictions.show(10)

+-----------+----------+------+--------------+---------------+----------+
|customer_id|product_id|rating|product_id_idx|customer_id_idx|prediction|
+-----------+----------+------+--------------+---------------+----------+
|         10|    416911|   4.0|         203.0|        22928.0|  4.107484|
|         27|  23503622|   5.0|         309.0|        44599.0|  4.593385|
|         36|    418636|   5.0|         227.0|        17556.0| 4.5891676|
|         50|  35580965|   4.0|         402.0|        46035.0| 4.0528626|
|         51|   8141868|   5.0|           4.0|       188972.0| 4.6610713|
|         83|    511328|   5.0|         321.0|        56438.0|  4.657878|
|        158|  25078249|   4.0|          51.0|         1547.0| 4.4927764|
|        277|  71198812|   5.0|         862.0|         4657.0|  4.774817|
|        277|  71201117|   5.0|        1199.0|         4657.0| 4.6615915|
|        330|  25918805|   5.0|         289.0|         8525.0| 4.4783545|
+-----------+----------+------+-------